# Bookkeeping

In [9]:
import numpy as np
import pandas as pd
import itertools

In [12]:
from sedgen.input import Input

In [19]:
dataset = Input(["Q" ,"P", "K", "B", "O", "A"])

In [59]:
minerals_csd = dataset.set_random_grain_size_distributions()

In [51]:
from scipy.stats import lognorm

In [60]:
lognorm.rvs(minerals_csd[0])

TypeError: '>' not supported between instances of 'rv_frozen' and 'int'

In [58]:
np.vectorize(lambda x: lognorm.rvs(x, 100))(minerals_csd)

TypeError: '>' not supported between instances of 'rv_frozen' and 'int'

In [41]:
dataset.get_random_csd_sample(minerals_csd)

AttributeError: 'numpy.ndarray' object has no attribute 'rvs'

In [17]:
dataset.get_interface_labels()

['QQ',
 'QP',
 'QK',
 'QB',
 'QO',
 'QA',
 'PP',
 'PK',
 'PB',
 'PO',
 'PA',
 'KK',
 'KB',
 'KO',
 'KA',
 'BB',
 'BO',
 'BA',
 'OO',
 'OA',
 'AA']

In [ ]:
np.histogram()

In [18]:
dataset.get_number_of_crystals()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [6]:
# Load jupyter extension to reload packages before executing user code.
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
# Reload all packages (except those excluded by %aimport) every time before executing the Python code typed.
%autoreload 2

## SedGen algorithms
____

**Assumptions**
- All grains are spherical  

**Relations**
- $X = Y + Z$

In [2]:
m = 6  # number of mineral species
d = 10 # number of size classes forming a geometric series
Y = np.zeros((m, d)) # (m x d) number of crystals forming part of polycrystalline grains (pcg)
Z = np.zeros((m, d)) # (m x d) number of crystals occurring as monocrystalline grains (mcg)
R = np.zeros((1, d)) # (1 x d) number of polycrystalline grains (pcg)

X = Y + Z # (m x d) number of crystals in parent rock

In [3]:
mineral_labels = ["Q" ,"P", "K", "B", "O", "A"]

In [4]:
interface_labels = ["".join(pair) for pair in itertools.combinations_with_replacement(mineral_labels, 2)]
interface_labels

['QQ',
 'QP',
 'QK',
 'QB',
 'QO',
 'QA',
 'PP',
 'PK',
 'PB',
 'PO',
 'PA',
 'KK',
 'KB',
 'KO',
 'KA',
 'BB',
 'BO',
 'BA',
 'OO',
 'OA',
 'AA']

In [ ]:
interface_labels

## Mechanical weathering
____

In [ ]:
# Selection for breakage
# D = size of grain
# alpha_Qm = susceptibility to breakage normallized to monocrystalline quartz (Qm)
alpha_Qm = np.zeros((m, 1)) # CONSTANT
# delta_k = model time step
delta_k = int
# gamma = coefficient of time-step dependent strength of pcg # CONSTANT
gamma = float
# alpha = susceptibility to breakage
alpha = alpha_Qm + alpha_init * delta_k ** gamma
# z = abundance in the grain population
S = D * alpha * z
# probability of breakage ([m + 1] x d)
P = S / np.sum(S)

In [4]:
# Example values
mineral_constants = pd.DataFrame({"alpha_Qm": [1, 1.5, 1.2, 2, 3, 1.6]}, index=mineral_labels)
mineral_constants

,alpha_Qm
Q,1.0
P,1.5
K,1.2
B,2.0
O,3.0
A,1.6


### Inter-crystal breakage (of pcg)

In [27]:
# beta = (g x 1) susceptibility of mineral interface to breakage
interface_constants = pd.DataFrame({"beta_QQ": [(alpha[0] * alpha[1]) / 2 for alpha in itertools.combinations_with_replacement(mineral_constants["alpha_Qm"], 2)]}, index=interface_labels)
interface_constants

,beta_QQ
QQ,0.500
QP,0.750
QK,0.600
QB,1.000
QO,1.500
QA,0.800
PP,1.125
PK,0.900
PB,1.500
PO,2.250


### Intra-crystal breakage